In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from tqdm import tqdm

folder = 'src'

if os.getcwd() != folder:
    os.chdir(folder)

In [6]:
name2id_old = json.load(open("name2id.json"))
name2probs_old = json.load(open("name2probs.json"))

contests_in_old = name2id_old.values()

In [7]:
cf_duration_api = requests.get("https://codeforces.com/api/contest.list?gym=false")
contests = cf_duration_api.json()
cf_duration_dataframe = pd.json_normalize(contests,['result'])

cf_duration_dataframe = cf_duration_dataframe[cf_duration_dataframe["relativeTimeSeconds"] > -3000]
cf_duration_dataframe = cf_duration_dataframe[~cf_duration_dataframe["id"].isin(contests_in_old)].reset_index(drop=True)
cf_duration_dataframe

,id,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds


In [8]:
id2name = {}
for row in cf_duration_dataframe.values:
    id2name[row[0]] = row[1]

In [9]:
id2probs = {}
for contest in tqdm(id2name, total=len(id2name)):
    try:
        page = "https://codeforces.com/api/contest.standings?contestId=" + str(contest) + "&from=1&count=1&showUnofficial=false"
        cf_submissions_api = requests.get(page)
        submissions = cf_submissions_api.json()
        #cf_dataframe = pd.json_normalize(submissions,['result'])
        
        standings = []
        problems = sorted([x["index"] + " - " + x["name"] for x in submissions["result"]["problems"]])

        id2probs[contest] = problems
    except:
        print(f"ERROR {contest}")

0it [00:00, ?it/s]


In [10]:
name2id = {v:k for k,v in id2name.items()}

name2probs = {}
for x in id2name:
    if x in id2probs:
        name2probs[id2name[x]] = id2probs[x]

In [12]:
name2id.update(name2id_old)
name2probs.update(name2probs_old)

In [13]:
with open("name2id.json", "w") as outfile:
    json.dump(name2id, outfile)
    
with open("name2probs.json", "w") as outfile:
    json.dump(name2probs, outfile)